In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_palette('tab10')
import statistics
import numpy as np
# from geopy import geocoders
import pycountry_convert as pc
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler
from sklearn import preprocessing
from sklearn.compose import make_column_selector as selector
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline

## Summary (Steps and Findings)

### Data Prepocessing

We were given the quality grading data of two different coffee species (Arabica and Robusta). To get the data ready for EDA and some modeling, we have applied the following preprocessing steps right in flow.
 
   1.  <b> Combining both dataset:</b> Arabica dataset was much bigger (1300+ instances) than the Robusta one (only 26 records). Therefore, it was better to combine both of the. Through analyzing some statistical metrics (i.e. central tendencies), we saw that both of the datasets are not that different. Hence, we appended the Robusta dataset with Arabica.
   
   2. <b> Dealing with Missing Values:</b> We notice that there are eight columns with missing values (0.07% to 17%): three columns with continuous (float) values, four categorical columns, and one integer column. We followed a univariate based missing values imputation using the mean/mode of the column within the ('country_of_origin', 'species') group.
   
   3. <b> Selecting important features:</b> In order to find relevant features, we computed correlation between target variable (quality_score) and other numerical features. We inferred that columns ('quality_score', 'aroma', 'flavor', 'aftertaste', 'acidity', 'body', 'balance', 'clean_cup', 'sweetness', 'cupper_points') are highly correlated with the quality_score, while columns ('moisture', 'category_one_defects', 'category_two_defects') are negatively correlated with our target feature quality_score. We kept both of these sets and dropped other numerical columns
   
   4. <b> Adding new column: </b> As we are particularly interested to do some analysis on regional bases (i.e. Central America). Therefore, we add a new column as 'continent'. Although, Central America is not a separate continent yet we kept it separate just for our analysis.
   
   4. <b> Feature Scaling and Encoding: </b> Most of our numerical features were not on a single scale which might hinder the model to converge fast. Moreover, After feature selection, we are also left with some important categorical columns ('species','country_of_origin','variety','processing_method','color'). We applied MinMaxScaler to scale numeric features and OneHotEncoder to encode the categorical features. However, we used this part of preprocessing in the prediction pipeline defined in the Modeling section.
   
### Exploratory Data Analysis

 We did some EDA, to find the answers to the questions asked with the challenge,

   1. <b>Understand which factors contribute to a high coffee quality score.</b>
   
   Using the Data Preprocessing and EDA we found the following contributing factors (especially Fig 2, EDA, Fig 3, and regression coefficients)
   
       1. Positive factors: ('quality_score', 'aroma', 'flavor', 'aftertaste', 'acidity', 'body', 'balance', 'clean_cup', 'sweetness', 'cupper_points') with super high to high correlation. While in categorical variable ('continent', 'country_of_origin','variety') are also highly important.
       2. Negative factors: ('moisture', 'category_one_defects', 'category_two_defects') are factors, negatively impact the quality score. Here, 'category_one_defects' and 'category_two_defects' have a medium impact while moisture has a comparatively lower impact.
       https://www.coffeestrategies.com/wp-content/uploads/2020/08/Green-Coffee-Defect-Handbook.pdf
       
   2. <b>Provide a recommendation to a prospective coffee farmer on what to consider to produce a high-quality coffee in Central America.</b>

   A major chunk of the records (~44%) are associated with the Central America region. Here, we can infer that the countries in Central America are the biggest producer of coffee. However, we notice on average Central American countries have a lower quality score. We also notice that mostly in the case of highly correlated features, 'Central America' is performing comparatively poor than other continents and better on the negatively correlated features, especially category_two_defect and moisture. Moreover, we notice that the distribution of values for 'Central America' in each feature is quite wide along with having many outliers. Which indicated a sort of inconsistency in production. 
   
   To find the reasons behind this, there could be three factors behind this: 1) variety, 2) any specific country within the region with poor performance, or 3) processing_method. We investigated this draft our suggestions as
          
        1. The major chunk of production comprises three varieties (Bourban, Typica, and Caturra). We can notice that 'Typica' (holding the second highest chunk) has an inferior score but the highest defect score as compared to the third one 'Catura' and all the varieties in the top 5. So, the first suggestion we can make is to drop Typica or get it least production.
        2. Drop 'Typica' or at least decrease its production as compared to the 'Catura' (which is also best in the rest of the regions). 'Catura' can give them a better score with less defect score. Moreover, we have some examples from the coffees produced in small quantities but people liked them very much. May take look at them if it would worthful to promote them more e.g. 'Gesha'.
        3. We see that about 72% of the production comes from Mexico (41%) and Guatemala (31%). However, Mexico has a slightly lower score than Guatemala but more importantly very high score for negatively correlated features like moisture, category_one_defect, and category_two_defect. 
        4. The main reason behind the Mexican farmers getting lower scores is the variety 'Typica'. We already saw that 'Typica' has more defects and higher moisture level than most of the other which make it a bit less attractive.
        5. The Farmers in Mexico should more focus on reducing the moisture and reducing defects. This will ultimately get them a better quality score.

### Prediction Model

To apply or prediction model, we using the following ingredients and steps

   1. <b>Data:</b> data prepared in the data preprocessing section We just add the feature scaling encoding function here along with the prediction pipeline.
   2. <b>Models: </b> We employed three regression-based model: 1) Linear Regression, 2) Decision Tree Regression, and 3) Random Forest Regression.
   3. <b>Evaluate:</b>
       1. Cross validate: We use five-fold cross-validation to evaluate our models.
       2. Metrics: We use R^2 and Mean Squared Error to evaluate the performance.
   4. <b>Results:</b> Among all the three models, linear regression performed better both in terms of R^2 and Mean Squared Error.
   
<b>Note:</b> I just chose the simple option. However, we can further apply more sophisticated data processing (dealing with outliers or applying power transformations to align data with respect to the normal distribution) or model tuning to get better results.


In [ ]:
# gn = geocoders.Google()
# gn.geocode("Mexico")

We are given with a data of two species 'Arabica' and 'Robusta'. Let's load both of them

In [ ]:
data_arabica = pd.read_csv('./data/arabica.csv')
data_robusta = pd.read_csv('./data/robusta.csv')

## 1. Data Preprocessing

### Combining both datasets

First let's explore both datasets and see if we can merge them together.

In [ ]:
data_arabica.head(3)

In [ ]:
data_arabica.describe()

In [ ]:
data_arabica.info()

In [ ]:
data_robusta.head(3)

In [ ]:
data_robusta.describe()

In [ ]:
data_arabica.describe()

We saw that statistical features of both datasets are not that different. Meanwhile, data_robusta is not that big that we can build that modely solely on that dataset. Let's append(merge at the end) data_robusta with the data_arabica and use that dataset for further analysis

In [ ]:
data_robusta = data_robusta[data_arabica.columns]

In [ ]:
data = data_arabica.append(data_robusta).reset_index(drop = True)

In [ ]:
data.columns = map(str.lower, data.columns)

In [ ]:
data.describe()

So we can notice that even after ccombining the data from both the species statistical features dont change much except a few points in some features.

In [ ]:
data[data.country_of_origin == 'USA'].region.unique()

In [ ]:
# data = pd.get_dummies(data, columns = ['species'])

### Missing values imputations

Let's check the missing values

In [ ]:
data.info()

For the ease of usage get the percentage of missing values in each column

In [ ]:
(data.isnull().sum()/data.shape[0])*100

We notice that there are eight columns with missing values (0.07% to 17%): three columns with continous (float) values, four categorical columns, and one is integer column. We might not need some of these columns at the end, yet first deal with the missing values. Dealing with missing values is always depends on the and the ratio of missing values itself. Normally, if a columns has missing values less than 50% then we can try to deal with that. Since, in all of the columns we have missing values less than even 20% so let's impute them.

Well, there three main categories of techinques to deal with the missing values.

   1. Univariate
   2. Hybrid
   3. Multivariate
   
In univariate, we normally impute the missing values using central tendency based metrics, for instance, Mean or Median (in case of numeric columns) and Mode (in case of categorical columns).

Hybrid is a simple group based univariate imputation. Make groups (using group by) within the data and apply univariate imputation within that. 

In case of multivariate we can use advance or more sophisticated techniques like KNNImputer, Iterative Imputer etc.

Here, we will use the Hybrid approach to impute to missing values.  

We can divide the whole data based on two main groups 'species' and 'country_of_origin'. Therefore, let's make these groups and use the missing values imputer within that group

In [ ]:
#data.groupby(by=['country_of_origin','species']).count()

In [ ]:
data = data.fillna(np.nan)

In [ ]:
missing_columns = data.columns[data.isnull().any()].tolist()

In [ ]:
data[missing_columns].isnull().sum()

Let's deal each column one-by-one and fill the missing values in three steps.

- First, we will fill the missing values with mean/mode of the colum within ['country_of_origin','species'] group. 
- If we are still left with the missing values then we fill those only with ['country_of_origin'] group.
- If we are still left then we can drop those records.

In [ ]:
def fill_missing_values_multi_columns(data, missing_columns):
    for col in missing_columns:
        if data[col].dtype=="object":
            data[col].fillna(data.groupby(['country_of_origin','species'])[col].transform(statistics.mode), inplace = True)
#             data[col] = data.groupby(['country_of_origin','species'])[col].transform(lambda x: x.fillna(statistics.mode(x)))
        if data[col].dtype=="int64" or data[col].dtype=='float64':
#             data[col] = data.groupby(['country_of_origin','species'])[col].transform(lambda x: x.fillna(x.mean()))                                                                      
            data[col].fillna(data.groupby(['country_of_origin','species'])[col].transform('mean'), inplace = True)
#         if data[col].dtype=='float':
#             data[col] = imp.fit_transform(data[col].values.reshape(-1,1))
    return data

def fill_missing_values_single_column(data, missing_columns):
    for col in missing_columns:
         if data[col].dtype=="object":
#             data[col].fillna(data.groupby(['country_of_origin','species'])[col].transform(statistics.mode), inplace = True)
            data[col] = data.groupby(['country_of_origin'])[col].transform(lambda x: x.fillna(statistics.mode(x)))
         if data[col].dtype=="int64" or data[col].dtype=='float64':
            data[col] = data.groupby(['country_of_origin'])[col].transform(lambda x: x.fillna(x.mean()))                                                                      
            #combi[col].fillna(combi[col].mode()[0], inplace=True)
#              data[col].fillna(data.groupby(['country_of_origin','species'])[col].transform(statistics.median), inplace = True)
#         if data[col].dtype=='float':
#             data[col] = imp.fit_transform(data[col].values.reshape(-1,1))
    return data

In [ ]:
data = fill_missing_values_multi_columns(data.copy(), missing_columns)

In [ ]:
data[missing_columns].isnull().sum()

We saw that even after imputing based 'country_of_origin' and 'species' still we are left with missing values. Let's check that scenario with couple of columns

Let's do another iteration

In [ ]:
data = fill_missing_values_multi_columns(data.copy(), missing_columns)

In [ ]:
data[missing_columns].isnull().sum()

We notice that within that species all the values are null for region column of that country of origin. 

In [ ]:
data = fill_missing_values_single_column(data.copy(), missing_columns)

In [ ]:
data[missing_columns].isnull().sum()

In [ ]:
# data[data.isnull().any(axis=1)]

In [ ]:
data = data.dropna()
data.reset_index(drop = True, inplace = True)

In [ ]:
data[missing_columns].isnull().sum()

### Feature Selection and Transformation

We cannot use all of the given feature columns for our final modelling. By looking at the data we can simply drop some of the columns (e.g. status, owner). Let's drop these columns first.

In [ ]:
data['status'].unique()

status value remains constant

In [ ]:
data['owner'].nunique()

With an abstract level thinking we can surely say that these columns wont be helpul for the rest of the analysis. Let's drop them here.

In [ ]:
data = data.drop(columns=['status','owner'])

In [ ]:
data.head(3)

 Lets evaluate rest of feature columns and see if we can drop that or no. 

In [ ]:
data[data.country_of_origin == 'India'].region.unique()

In [ ]:
data[data.country_of_origin == 'USA'].region.unique()

we may find plenty of the cases where region is same but written with incorrect spellings. In another case a region of India 'CHIKMAGALUR' is is given as a region 'USA'. Therefore, we cannot trust this column as well. One option could be to apply some advanced NLP-based techniques to stream line these anomalies. However, its seems we dont need this column for the high level analysis. Let's drop this one as well.

In [ ]:
data = data.drop(columns=['region'])

In [ ]:
data.head(3)

In [ ]:
data['color'].unique()

In [ ]:
data['variety'].unique()

In [ ]:
data['processing_method'].unique()

So, we are only left with five categorical feature columns (species, country_of_origin, variety, processing_method, color). These columns perhaps would be help to answer the questions given with the challenge and more importantly to build predictive model.

Let's also check if we need time based colums like 'grading_year' or 'harvest_year'?

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 3))
axes = axes.ravel()

sns.boxplot(x='grading_year',y='quality_score', data = data , ax = axes[0])
sns.boxplot(x='harvest_year',y='quality_score', data = data , ax = axes[1])

fig.tight_layout()

Since year is also a time based but a numeric featues so we can some put that in correlation matrix as well. Let's see how it would behave

Let's have a look at the non-categorical columns to see if they have any kind of correlation with the target colum

In [ ]:
data_num = data.select_dtypes(include=[np.float64, np.int64])#.drop(columns = ['harvest_year','grading_year'])

Let's see we can find something from correlation heatmap

In [ ]:
f, ax = plt.subplots(figsize=(15, 10))
sns.heatmap(data_num.corr(), annot=True)
plt.title('Fig-2: Correlations', fontsize = 20)
plt.show()

<b>Inference 1: </b>We can notice columns ('quality_score', 'aroma', 'flavor', 'aftertaste', 'acidity', 'body',
       'balance', 'clean_cup', 'sweetness', 'cupper_points') are highly correlated with the quality_score, columns (
       'quakers',
       'altitude_numeric', 'harvest_year','grading_year') doesnt provide much information regarding the quality_score, while columns ('moisture',
      'category_one_defects', 'category_two_defects') are negatively correlated with our traget feature quality_score. Here, we also notice the higher the defect a coffee has lower the score.

Since ('quakers', 'altitude_numeric') do not provide much information so let's drop them. Let's keep the negatively correlated features for a while. Later on, we will see.

In [ ]:
data_num.drop(columns = ['quakers', 'altitude_numeric'], inplace = True)
data.drop(columns = ['quakers', 'altitude_numeric'], inplace = True)

We will keep the years based features just for the purpose of some analysis. We will drop them just before modeling

In [ ]:
data_num.columns

### Adding Continent column

last but not the least. since we need to do some analysis based on continent central_america so lets add a column to identify if the country is from central america or not

In [ ]:
data['country_of_origin'] = data['country_of_origin'].apply(lambda x: x.split(' ')[0] if x.split(' ')[0] == 'USA' else x)

In [ ]:
def country_to_continent(country_name):
    country_alpha2 = pc.country_name_to_country_alpha2(country_name)
    country_continent_code = pc.country_alpha2_to_continent_code(country_alpha2)
    country_continent_name = pc.convert_continent_code_to_continent_name(country_continent_code)
    return country_continent_name

In [ ]:
data.country_of_origin.unique()

In [ ]:
central_america = ['Mexico','Guatemala','Honduras','Nicaragua','El Salvador','Costa Rica','Panama','Belize']
cont = ['Central America' if country in central_america else country_to_continent(country) for country in data.country_of_origin.tolist()]

In [ ]:
data['continent'] = cont

### Feature Scaling and Encoding

We already noticed that distributions of our numeric features are not on a single scale which might hinder the model to converge quickly. Therefore, we have to apply scaling to make to in a single range. We can apply this scaling here or add the scaling and encoding functions in the prediction pipe line. It would be more organized to keep it with the prediction pipeline.

However, if we want to do it here then uncomment the script given belowSo, Let's apply simple features scaling.

Let's first create a copy of data which we will use for modeling. The original copy will be used for EDA.

In [ ]:
# transformed_data = data.copy()

# num_cols = list(data.select_dtypes(include=['int64','float64']))
# num_cols.remove('quality_score')
# num_cols.remove('grading_year')
# num_cols.remove('harvest_year')

# scaler = MinMaxScaler()

# transformed_data[num_cols] = scaler.fit_transform(transformed_data[num_cols])

We are not interested to scale target variable or year columns. All the numeric columns exluding target column ('quality_year') and years columns need to be removed.

In case we want to use categorical variable then we have encode them as numeric column but it cant be pure numeric. the best way to encode them is to use label-encoder or one-hot-encoder. It would be more flexible to use that along with the prediction pipeline

In [ ]:

# encoders = {}

# transformed_data = data.copy()

# cat_columns =  ['species', 'country_of_origin', 'variety', 'processing_method', 'color','altitude_uom']

# le = preprocessing.LabelEncoder()

# transformed_data[cat_columns] = le.fit_transform(transformed_data[cat_columns])

# for col in cat_columns:
    
#     le = preprocessing.LabelEncoder()
    
#     transformed_data[col] = le.fit_transform(data[col])
    
#     encoders[col] = le

Our transformed data is ready for modeling

## 2. Exploratory Data Analysis

In [ ]:
data.head(3)

We are interested to see formation from the very high level like continent. So, let's start with that.

In [ ]:
((data['continent'].value_counts()/len(data['continent']))*100).sort_values(ascending = False).plot.pie()#.plot(kind = 'bar')
#plt.ylabel('Percentage')
#plt.xlabel('Continents')
plt.show()

In [ ]:
((data['continent'].value_counts()/len(data['continent']))*100).sort_values(ascending = False).plot(kind = 'bar')
plt.ylabel('Percentage')
plt.xlabel('Continents')
plt.show()

We have 45%  of the cases are relevant to the contries in central americal. In other words we can say that these countries have produceed about 45% of the whole production.

Let's the distribution of quality_score wrt to each continent

In [ ]:
# data.groupby('continent')['quality_score'].median().reset_index().plot(kind = 'bar')

In [ ]:
ax = sns.boxplot(x="continent", y="quality_score", data=data)
plt.xticks(rotation = 45)
plt.show()

As per the distribution, we notice that Central America has a wider dsitribution with some outliers. Moreover, it has lower median value than the quality scores from other continents. 

We need to analyze on the core factors behind the quality score. In Fig-1, we already saw that quality_score is highly correlated with ('aroma', 'flavor', 'aftertaste', 'acidity', 'body', 'balance', 'clean_cup', 'sweetness', 'cupper_points') while negatively correlated with columns ('moisture', 'category_one_defects', 'category_two_defects'). So let's visualize and see the the continent wise perfromance wrt to each feature

In [ ]:
fig, axes = plt.subplots(5, 3, figsize=(15, 15))
axes = axes.ravel()

cols = ['aroma', 'flavor', 'aftertaste',
       'acidity', 'body', 'balance', 'clean_cup', 'sweetness', 'cupper_points',
       'moisture', 'category_one_defects', 'category_two_defects']



for axs, col in zip(axes, cols):
    plot = sns.boxplot(x='continent', y=col, data=data, ax = axs)
    _ = plot.set(title='({})'.format(col))
    axs.tick_params(labelrotation=45)
    plt.xticks(rotation = 45)
axes[-3].set_visible(False)
axes[-2].set_visible(False)
axes[-1].set_visible(False)
#fig.title('month-wise sales distribution of each season', fontsize=16)
fig.tight_layout()



In [ ]:
cont_data_agg = data.groupby(['continent'])[['quality_score', 'aroma', 'flavor', 'aftertaste', 'acidity', 'body',
       'balance', 'clean_cup', 'sweetness', 'cupper_points', 'moisture',
       'category_one_defects', 'category_two_defects']].mean().reset_index()

In [ ]:
cont_data_agg.sort_values(by='quality_score', ascending = False).style.background_gradient(cmap='Blues')

We notice that mostly in case of highly correlated features, 'Central America' is performing compartively poor than other continents and better on the negatively correlated features especially category_two_defect and moisture. Moreover, another we notice that the distribtion of values for 'Central America' in each feature is quite wide along with having many outliers. Which indicated a sort of inconsistency in production. This inconistency may be due to the variation from different countries in this region. We will see that later on. 

Potentially, there could be three factors behind this: 1) variety, 2) any specific country within region with poor performance, or 3) processing_method. Let's investigate both of these aspects and compare them with rest of the world.

#### Varieties

In [ ]:
CA_data = data[data.continent == 'Central America'].reset_index(drop = True)

In [ ]:
# CA_data

First let's invetigate the quality of all varieties the framers in Central America producing.

In [ ]:
CA_variety_agg = CA_data.groupby('variety')['quality_score'].count().reset_index()#
CA_variety_agg.rename(columns = {'quality_score':'count'}, inplace = True)
CA_variety_agg.sort_values(by = 'count', ascending = False, inplace = True)

In [ ]:
# fig, axes = plt.subplots(1, 2, figsize=(10, 5))
# axes = axes.ravel()

sns.barplot(x='variety', y='count', data=CA_variety_agg, palette = sns.color_palette('tab10'))

# sns.barplot(x='variety', y='mean', data=CA_variety_agg, ax=axes[1], palette = sns.color_palette('tab10'))
# axes[1].tick_params(labelrotation=45)
#fig.tight_layout()
plt.xticks(rotation = 45)
plt.show()

In [ ]:
CA_variety_agg_all = CA_data.groupby('variety')[['quality_score', 'aroma', 'flavor', 'aftertaste', 'acidity', 'body',
       'balance', 'clean_cup', 'sweetness', 'cupper_points', 'moisture',
       'category_one_defects', 'category_two_defects']].mean().reset_index()
CA_variety_agg = CA_variety_agg.merge(CA_variety_agg_all, how = 'inner', on = 'variety').sort_values(by = ['count','quality_score'], ascending = False)
CA_variety_agg.style.background_gradient(cmap='Blues')

The major chunk of production comprises on three varieties (Bourban,Typica, and Caturra). We can notice that 'Typica' holding the second highest chunk has inferior score but highest defect score as compared to the third one 'Catura' and infact all the varieties in top 5. So, first suggestion we can make is to drop Typica or get it least production. 

<b> Inference-2: </b> Drop 'Typica' or at least decrease its production as compared to the 'Catura'. 'Catura' can give them better score with less defect score. Moreover, we have some examples from the coffies produced in small quantities but people liked them very much. May take look at them if it would worthfull to promote them more e.g. 'Gesha'.

Let's see if we can get something by comparing our inference with rest of regions.

In [ ]:
data_variety_agg = data.groupby('variety')['quality_score'].count().reset_index()#
data_variety_agg.rename(columns = {'quality_score':'count'}, inplace = True)
data_variety_agg.sort_values(by = 'count', ascending = False, inplace = True)

In [ ]:
data_variety_agg_all = data.groupby('variety')[['quality_score', 'aroma', 'flavor', 'aftertaste', 'acidity', 'body',
       'balance', 'clean_cup', 'sweetness', 'cupper_points', 'moisture',
       'category_one_defects', 'category_two_defects']].mean().reset_index()
data_variety_agg = data_variety_agg.merge(data_variety_agg_all, how = 'inner', on = 'variety').sort_values(by = ['count','quality_score'], ascending = False)
data_variety_agg.head(15).style.background_gradient(cmap='Blues')

In [ ]:
# data_variety_agg = data.groupby('variety')['quality_score','flavor','aftertaste','balance','cupper_points'].mean().reset_index().sort_values(by = ['quality_score','flavor','aftertaste','balance','cupper_points'], ascending = False)

In [ ]:
# data_variety_agg

In [ ]:
data_variety_agg = data.groupby('variety')['quality_score'].agg(['mean','count']).reset_index().sort_values(by = 'count', ascending = False)

In [ ]:
sns.barplot(x='variety', y='count', data=data_variety_agg, palette = sns.color_palette('tab10'))
plt.xticks(rotation = 90)
plt.show()

By looking at the other regions we can verify or Inference-2: Increasing the production of 'Catura' and decreasing the 'Typica'. Moreover, try new varaities like 'Gesha' or 'Yellow Bourbon'

##### Countries

In [ ]:
CA_country_agg = CA_data.groupby('country_of_origin')['quality_score'].count().reset_index()#
CA_country_agg.rename(columns = {'quality_score':'count'}, inplace = True)
CA_country_agg.sort_values(by = 'count', ascending = False, inplace = True)

In [ ]:
# fig, axes = plt.subplots(1, 2, figsize=(10, 5))
# axes = axes.ravel()

sns.barplot(x='country_of_origin', y='count', data=CA_country_agg, palette = sns.color_palette('tab10'))

# sns.barplot(x='variety', y='mean', data=CA_variety_agg, ax=axes[1], palette = sns.color_palette('tab10'))
# axes[1].tick_params(labelrotation=45)
#fig.tight_layout()
plt.xticks(rotation = 45)
plt.show()

In [ ]:
CA_data.shape

In [ ]:
CA_country_agg_all = CA_data.groupby('country_of_origin')[['quality_score', 'aroma', 'flavor', 'aftertaste', 'acidity', 'body',
       'balance', 'clean_cup', 'sweetness', 'cupper_points', 'moisture',
       'category_one_defects', 'category_two_defects']].mean().reset_index()
CA_country_agg = CA_country_agg.merge(CA_country_agg_all, how = 'inner', on = 'country_of_origin').sort_values(by = ['count','quality_score'], ascending = False)
CA_country_agg.style.background_gradient(cmap='Blues')

We see that about 72% of the production comes from Mexico (41%) and Guatemala (31%). However, Mexico has slighty lower score than Guatemala but more importanly very high score for negative correllated features like moisture, category_one_defect, and category_two_defect. The Farmers in Mexico should more focus on to reduce the moisture and reduce the defects. This will ultimately get them better quality score.

In [ ]:
Mexico_variety = CA_data[CA_data.country_of_origin == 'Mexico']['variety'].value_counts().reset_index()

In [ ]:
sns.barplot(x='index',y='variety', data = Mexico_variety)
plt.xticks(rotation = 45)
plt.show()

The main reason behind the Mexico farmers getting lower scores is the variety 'Typica'. We already saw that 'Typica' has more defects and higher moisture level than most of the other which makes it bit less attractive.

In [ ]:
# fig, axes = plt.subplots(5, 3, figsize=(15, 10))
# axes = axes.ravel()

# cols = ['quality_score', 'aroma', 'flavor', 'aftertaste',
#        'acidity', 'body', 'balance', 'clean_cup', 'sweetness', 'cupper_points',
#        'moisture', 'category_one_defects', 'category_two_defects']



# for axs, col in zip(axes, cols):
#     plot = sns.barplot(x='continent', y=col, data=cont_data_agg, ax = axs)
#     _ = plot.set(title='({})'.format(col))
# axes[-2].set_visible(False)
# axes[-1].set_visible(False)
# #fig.title('month-wise sales distribution of each season', fontsize=16)
# fig.tight_layout()



In [ ]:
CA_data[CA_data.country_of_origin == 'Mexico']['processing_method'].value_counts()

In [ ]:
CA_data[CA_data.country_of_origin == 'Guatemala']['processing_method'].value_counts()

In [ ]:
data[data.continent == 'Central America']['processing_method'].value_counts()

In [ ]:
data['processing_method'].value_counts()

In [ ]:
data[data.continent == 'Central America']['variety'].value_counts()

In [ ]:
data[data.continent == 'Africa']['variety'].value_counts()

Let's transform these features to find the relationship with out dependent feature (quality_score)

## 3. Modeling

Since records for both species are clustered togther so let's shuffle the data and seperate the data and target variable

In [ ]:
data = data.sample(frac=1).reset_index(drop=True)

In [ ]:
fig, axes = plt.subplots(5, 3, figsize=(15, 10))
axes = axes.ravel()


cols = ['aroma', 'flavor', 'aftertaste',
       'acidity', 'body', 'balance', 'clean_cup', 'sweetness', 'cupper_points',
       'moisture', 'category_one_defects', 'category_two_defects']


for axs, col in zip(axes, cols):
    plot = sns.regplot(x='quality_score', y=col, data=data, ax = axs)
    _ = plot.set(title='({})'.format(col))
axes[-3].set_visible(False)
axes[-2].set_visible(False)
axes[-1].set_visible(False)
fig.suptitle('Fig-3: Linear Relationship')
fig.tight_layout()

In [ ]:
X = data.drop(columns=['quality_score'])
Y = data['quality_score']

### Feature scaling and encoding processor

Instead doing features scaling in the preprocessing phase, we opted to keep it more simple and put that along the prediction pipeline

In [ ]:
numerical_columns_selector = selector(dtype_exclude=object)
categorical_columns_selector = selector(dtype_include=object)

In [ ]:
numerical_columns = numerical_columns_selector(X)
categorical_columns = categorical_columns_selector(X)

In [ ]:
numerical_columns

In [ ]:
numerical_columns.remove('grading_year')
numerical_columns.remove('harvest_year')

In [ ]:
categorical_columns.remove('altitude_uom')
categorical_columns.remove('continent')

We would like to check what would be the performance of model by including or exlcluding the important categorical features. Therefore, first we define the pipeline with categorical features and then later on we will train the model only uisng numeric variable.

In [ ]:
categorical_preprocessor = OneHotEncoder(handle_unknown="ignore")
numerical_preprocessor = MinMaxScaler()

In [ ]:
comb_preprocessor = ColumnTransformer([
    ('one-hot-encoder', categorical_preprocessor, categorical_columns),
    ('minmax_scaler', numerical_preprocessor, numerical_columns)])

### Prediction Pipeline

In [ ]:
linear_model = make_pipeline(comb_preprocessor, LinearRegression())
tree_model = make_pipeline(comb_preprocessor, DecisionTreeRegressor())
forest_model = make_pipeline(comb_preprocessor, RandomForestRegressor(random_state=0))

In [ ]:
# X = transformed_data.drop(columns=['quality_score','continent','harvest_year','grading_year','species', 'country_of_origin', 'variety', 'processing_method', 'color','altitude_uom'])
# Y = transformed_data['quality_score']

#### Evaluation

In [ ]:
scores = cross_validate(linear_model, X, Y, cv=5, scoring=['r2','neg_mean_squared_error'])
scores

In [ ]:
scores = cross_validate(tree_model, X, Y, cv=5, scoring=['r2','neg_mean_squared_error'])
scores

In [ ]:
scores = cross_validate(forest_model, X, Y, cv=5, scoring=['r2','neg_root_mean_squared_error'])
scores

<b> Only numeric features: </b> Now let's use only numeric features (identified using the Fig-2) to fit our model and see how it behaves.

In [ ]:
preprocessor = ColumnTransformer([('minmax_scaler', numerical_preprocessor, numerical_columns)])

In [ ]:
linear_model = make_pipeline(preprocessor, LinearRegression())
tree_model = make_pipeline(comb_preprocessor, DecisionTreeRegressor())
forest_model = make_pipeline(preprocessor, RandomForestRegressor(random_state=0))

In [ ]:
scores = cross_validate(linear_model, X[numerical_columns], Y, cv=5, scoring=['r2','neg_root_mean_squared_error'])
scores

In [ ]:
scores = cross_validate(forest_model, X[numerical_columns], Y, cv=5, scoring=['r2','neg_root_mean_squared_error'])
scores

### Result

Finally, we saw the linear regression (with both categorical and numerical features) has performed best among all the regression based model. 

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(
# ...     X, Y, test_size=0.20, random_state=42)
# lr = LinearRegression().fit(X_train[numerical_columns], y_train)
# lr.score(X_train[numerical_columns], y_train)
# numerical_columns

In [ ]:
# lr.coef_